### This code selects the sample of 94 FRBs used in the analysis of Gupta et al. (2025)
#### Please do not use the fluences from the file generated at the end. In our code implementation, correct fluences are always extracted by cross-referencing with the baseband catalog.

In [1]:
import pandas as pd
import numpy as np

In [5]:
def keep_nth_subburst(cat):
    """
    the catalog 1 analysis was done with the nth sub-burst for each source
    the public version of catalog 1 has all the sub-bursts separate, so we first keep
    
    logistically, we look at the sub_num array, which is an array of ints ranging
    from 0 to 5. if a 0 is immediately followed by a non-zero number (or sequence of numbers),
    we keep only the highest non-zero number in that sequence.
    `np.roll(arr, -1)` shifts the elements one position to the left.
    """
    sub_nums = cat['sub_num'].values
    indexes = np.where(np.roll(sub_nums, -1) == 0)[0]
    
    return cat.iloc[indexes]

def cut_cat_exposure(cat):
    """
    42 sub-bursts cut due to exposure issues
    these are marked with `excluded_flag = 1`, so we want
    all the bursts that have `excluded_flag = 0`
    """
    return cat[
        cat['excluded_flag'] == 0
    ]

def cut_far_sidelobe(cat):
    """
    3 sub-bursts cut because they are far-sidelobe detections
    their TNS names are 'FRB20190125B', 'FRB20190202B', 'FRB20190210D'
    """
    return cat[
        (cat['tns_name'] != 'FRB20190125B')
        & (cat['tns_name'] != 'FRB20190202B')
        & (cat['tns_name'] != 'FRB20190210D')
    ]

def cut_near_amb(cat):
    """
    cuts sub-bursts if the observed DM is within 1.5x the MW contribution
    (and always takes the more conservative MW contribution between ne2001 and ymw16)
    """
    dms_ne2001 = cat['dm_fitb'] - cat['dm_exc_ne2001']
    dms_ym16 = cat['dm_fitb'] - cat['dm_exc_ymw16']
    
    return cat[
        cat['dm_fitb'] >= 1.5 * np.maximum(dms_ne2001, dms_ym16)
    ]

def cut_low_snr(cat):
    """
    cuts sub-bursts if their signal-to-noise is < 12
    """
    return cat[
        cat['bonsai_snr'] >= 12
    ]

def cut_high_scat(cat):
    """
    cuts sub-bursts if their scattering timescale is > 10 ms
    """
    low_scats_ii = np.array([
        x for x in range(len(cat)) if
        ('<' not in cat['scat_time'].values[x] and float(cat['scat_time'].values[x]) < 10e-3)
        or ('<' in cat['scat_time'].values[x] and float(cat['scat_time'].values[x][1:]) < 10e-3)
    ])
    return cat.iloc[low_scats_ii]

def cut_repeat_bursts(cat):
    """
    mjd is time of arrivial with reference to ~400 MHz for specific sub-burst
    we assert that the catalog is sorted in time
    
    then we find the repeater names and keep only the first instance of each repeater
    """
    a = cat['mjd_400'].values
    assert np.all(a[:-1] <= a[1:])
    
    non_repeat_iis = np.array([x for x in range(len(cat)) if cat['repeater_name'].values[x] == '-9999'])
    
    repeater_iis = np.array([x for x in range(len(cat)) if cat['repeater_name'].values[x] != '-9999'])
    repeater_of = cat.iloc[repeater_iis]['repeater_name'].values
    keep_repeat_iis = np.array([list(repeater_of).index(elem) for elem in set(repeater_of)])
    
    keep_iis = np.sort(np.concatenate((non_repeat_iis, repeater_iis[keep_repeat_iis])))
    # keep_iis = np.sort(non_repeat_iis)
    
    return cat.iloc[keep_iis]

### Make CHIME Catalog 1 sample selection to get the 265 FRBs
#### The code block above and the one below are taken from https://github.com/kaitshin/CHIMEFRB-Cat1-Energy-Dist-Distrs/, and follow the sample used in CHIME Collaboration et al. (2021) analysis.

In [51]:
cat1_og = pd.read_csv('files/chimefrbcat1.csv')
print(f"the original catalog has {len(cat1_og)} (sub-)bursts")

cat1_og2 = keep_nth_subburst(cat1_og)
print(f"after keeping only the nth sub-burst of each source, the original catalog has {len(cat1_og2)} bursts")

cat1 = cut_cat_exposure(cat1_og2)
print(f"after exposure issue cuts, it has {len(cat1)} bursts")

cat1 = cut_far_sidelobe(cat1)
print(f"after far sidelobe cuts, it has {len(cat1)} bursts")

cat1 = cut_near_amb(cat1)
print(f"after near_AMB cuts, it has {len(cat1)} bursts")

cat1 = cut_low_snr(cat1)
print(f"after low SNR cuts, it has {len(cat1)} bursts")

# at this point, all sources have DM > 100
assert np.min(cat1['dm_fitb']) > 100

cat1 = cut_high_scat(cat1)
print(f"after high scattering cuts, it has {len(cat1)} bursts")

cat1i = cut_repeat_bursts(cat1)
print(f"after repeat burst cuts, it has {len(cat1i)} bursts")


the original catalog has 600 (sub-)bursts
after keeping only the nth sub-burst of each source, the original catalog has 536 bursts
after exposure issue cuts, it has 497 bursts
after far sidelobe cuts, it has 494 bursts
after near_AMB cuts, it has 470 bursts
after low SNR cuts, it has 298 bursts
after high scattering cuts, it has 270 bursts
after repeat burst cuts, it has 265 bursts


### Cross-reference with Baseband Catalog 1

In [53]:
cat1i = cat1i.reset_index()

cat2 = pd.read_csv('files/basecat1_catalog.csv')
cat1i_idx = []
cat2_idx = []
for i in range(cat2.shape[0]):
    flag = np.where(cat1i['tns_name'] == cat2['tns_name'][i])[0]
    if len(flag) != 0 and cat2['fluence'][i] > 0:
        cat1i_idx.append(flag[0])
        cat2_idx.append(i)

cat1i['repeater_name'][cat1i_idx][cat1i['repeater_name'][cat1i_idx] != '-9999']

112    FRB20190116B
116    FRB20190117A
Name: repeater_name, dtype: object

### Ensure that cross-referenced sample has the missed repeater burst

In [55]:
cat1 = cat1.reset_index()
cat2 = pd.read_csv('files/basecat1_catalog.csv')
cat1_idx = []
cat2_idx = []
for i in range(cat2.shape[0]):
    flag = np.where(cat1['tns_name'] == cat2['tns_name'][i])[0]
    if len(flag) != 0 and cat2['fluence'][i] > 0:
        cat1_idx.append(flag[0])
        cat2_idx.append(i)

cat1['repeater_name'][cat1_idx][cat1['repeater_name'][cat1_idx] != '-9999']

113    FRB20190116B
118    FRB20190117A
161    FRB20190222A
Name: repeater_name, dtype: object

In [58]:
## Really ensure that the bursts we are picking up are indeed the first repeater bursts in Baseband Catalog 1

print(cat1i['fluence'][cat1i_idx][cat1i['repeater_name'][cat1i_idx] != '-9999'])
print(cat1['fluence'][cat1_idx][cat1['repeater_name'][cat1_idx] != '-9999'])

112    3.7
116    9.9
Name: fluence, dtype: float64
113    3.7
118    9.9
161    8.6
Name: fluence, dtype: float64


In [64]:
## Print out the Baseband sample used in Gupta et al. (2025)
cat1.iloc[cat1_idx, 2:]

,tns_name,previous_name,repeater_name,ra,ra_err,ra_notes,dec,dec_err,dec_notes,gl,...,sp_idx_err,sp_run,sp_run_err,high_freq,low_freq,peak_freq,chi_sq,dof,flag_frac,excluded_flag
64,FRB20181213A,-9999,-9999,127.66,0.20,-9999,73.87,0.100,-9999,140.39,...,0.85,2.30,1.30,800.2,400.2,400.2,441492.962,440794,0.292,0
67,FRB20181214C,-9999,-9999,175.93,0.12,-9999,60.02,0.048,-9999,137.68,...,0.49,-0.08,0.75,800.2,400.2,400.2,454048.390,451737,0.274,0
68,FRB20181215B,-9999,-9999,254.81,0.17,-9999,47.56,0.030,-9999,73.62,...,0.87,-6.00,1.10,800.2,400.2,590.2,446582.692,439578,0.294,0
72,FRB20181219C,-9999,-9999,17.77,0.20,-9999,14.11,0.230,-9999,130.13,...,0.81,-4.60,1.60,735.5,400.2,400.2,434767.940,432281,0.306,0
75,FRB20181221A,-9999,-9999,230.58,0.20,-9999,25.86,0.210,-9999,39.52,...,2.40,-128.00,4.90,583.3,446.1,510.1,443691.859,440185,0.293,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,FRB20190628B,-9999,-9999,248.48,0.20,-9999,80.14,0.088,-9999,113.30,...,1.50,-0.40,3.60,583.5,400.2,400.2,365131.881,367225,0.410,0
265,FRB20190630D,-9999,-9999,143.36,0.20,-9999,8.90,0.250,-9999,224.58,...,1.30,-1.60,2.10,780.6,400.2,400.2,230876.012,229818,0.631,0
266,FRB20190701A,-9999,-9999,277.47,0.21,-9999,59.04,0.220,-9999,88.29,...,1.50,3.30,1.90,800.2,400.2,800.2,341779.300,341690,0.451,0
267,FRB20190701B,-9999,-9999,302.93,0.22,-9999,80.18,0.240,-9999,112.88,...,1.70,-11.80,3.10,732.8,400.2,471.5,329229.311,330137,0.470,0


In [65]:
### Save the sub-sample of 94 bursts. All data in the file is intensity data, and not baseband.
cat1.to_csv('files/chimefrbcutcat_94.csv', index=False)